In [ ]:
import os
import sys
import time
import random
import logging
from tqdm import tqdm
from pathlib import Path

from matplotlib import pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error, mean_squared_error

import lightgbm as lgb

%matplotlib inline

In [ ]:
!pip install optuna

In [ ]:
ROOT = Path.cwd().parent
INPUT = ROOT / "input"
DATA = INPUT / "tabular-playground-series-jan-2021"
WORK = ROOT / "working"

for path in DATA.iterdir():
    print(path.name)

In [ ]:
train = pd.read_csv(DATA / "train.csv")
test = pd.read_csv(DATA / "test.csv")
smpl_sub = pd.read_csv(DATA / "sample_submission.csv")
print("train: {}, test: {}, sample sub: {}".format(
    train.shape, test.shape, smpl_sub.shape
))

In [ ]:
train.target.describe()

In [ ]:
features = ['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7',
               'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14']


datax = pd.concat([train[features],test[features]], axis=0)
datax1 = datax.abs()


def featx(data, features):
    cx=int(1000000)
    for f in features:
        data[f] = data[f]*cx
        dmm = data[f].max()-data[f].min()
        data[f] = (data[f]-data[f].min())/dmm
        
    """px = []
    for row in data.iterrows():
        row = row*cx
        a = [row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],row[11],row[12],row[13],row[14]]
        sm= sum(a)
        mn= min(a)
        mx= max(a)
        md= np.med(a)
        sd= np.std(a)
        ma= np.mean(a)
        
        p0=mn/sm
        p1=mx/sm
        p2=md/sm
        p3=(mn+mx)/2
        p = [p0,p1,p2,p3]
        px.append(p)
        
    data['p0'] = px[0]
    data['p1'] = px[1]
    data['p2'] = px[2]
    data['p3'] = px[3]"""
    
    return data



data = featx(datax, features)
#data1 = featx(datax1, features)
data.head()

In [ ]:
data.shape

In [ ]:
X = data[:300000]
Xtest = data[300000:]
y = train['target']

"""X1 = data1[:300000]
Xtest1 = data1[300000:]"""

In [ ]:
"""
train = int(len(X)*0.9)
Xtrain, Xval = X.iloc[:train], X.iloc[train:]
ytrain, yval = y.iloc[:train], y.iloc[train:]



import optuna.integration.lightgbm as lgb

dtrain = lgb.Dataset(Xtrain, label=ytrain)
dval = lgb.Dataset(Xval, label=yval)
params = {"objective": "regression","metric": "rmse","verbosity": -1,"boosting_type": "gbdt","learning_rate":0.032,'feature_pre_filter': False}
model = lgb.train(params, dtrain, valid_sets=[dval], verbose_eval=100, early_stopping_rounds= 81 )
params = model.params
params
"""

In [ ]:
params={'objective': 'regression',
 'metric': 'rmse',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'learning_rate': 0.04,
 'feature_pre_filter': False,
 'min_data_in_leaf': 100,
 'lambda_l1': 9.486545871986378,
 'lambda_l2': 6.887206686817597,
 'num_leaves': 230,
 'feature_fraction': 0.4,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20}

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

N_FOLDS = 12

oof = np.zeros(len(y))
preds = np.zeros(len(Xtest))
params['learning_rate'] = 0.004
params['num_iterations'] = 12000
i=0
for train_ind, test_ind in tqdm(KFold(n_splits = N_FOLDS,shuffle=True,random_state=21).split(X),
                                total=N_FOLDS, desc="k-fold"):
    Xtrain = X.iloc[train_ind]
    Xval = X.iloc[test_ind]
    ytrain = y.iloc[train_ind]
    yval = y.iloc[test_ind]

    model = LGBMRegressor(**params)
    
    model.fit(Xtrain, ytrain,
              eval_set = ((Xval,yval)),
              early_stopping_rounds = 120,
              verbose = 1000)
    p = model.predict(Xval)
    oof[test_ind] = p
    
    preds += model.predict(Xtest)/N_FOLDS
    
    i+=1
    if i>5:
        print(f'MSE::: {np.round(mean_squared_error(y, oof, squared=False),8)}')
    else:
        pass

In [ ]:
submission = test[['id']]
submission['target'] = preds
submission.to_csv('submission.csv', index = False)